# 0. Import des packages et des données 

In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from helpers2 import S3Connection

In [4]:
from postal.expand import expand_address

In [5]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


In [ ]:
path1 = "DPE/DPE_logements.parquet"
DPE = s3.get_tables_from_s3(path1)

In [7]:
path_vf_2024 = "valeursfoncieres/vf_2024.csv"
vf_2024 = s3.read_csv_from_s3(path_vf_2024)

path_vf_2023 = "valeursfoncieres/vf_2023.csv"
vf_2023 = s3.read_csv_from_s3(path_vf_2023)

path_vf_2022 = "valeursfoncieres/vf_2022.csv"
vf_2022 = s3.read_csv_from_s3(path_vf_2022)

path_vf_2021 = "valeursfoncieres/vf_2021.csv"
vf_2021 = s3.read_csv_from_s3(path_vf_2021)

path_vf_2020 = "valeursfoncieres/vf_2020.csv"
vf_2020 = s3.read_csv_from_s3(path_vf_2020)

path_vf_2019 = "valeursfoncieres/vf_2019.csv"
vf_2019 = s3.read_csv_from_s3(path_vf_2019)

# 1. FONCTIONS UTILES

Normalisation des adresses du fichier DPE et valeurs foncières. 

In [8]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version de la normalisation
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [9]:
import re
def normalize_vf_address(address, code_postal):
    """ Normalise l'adresse et supprime l'arrondissement après 'PARIS' uniquement si le département est 75. """
    normalized_address = normalize_address(address)  

    if pd.notna(normalized_address) and str(code_postal).startswith("75"):
        # Supprime le numéro après "PARIS" uniquement pour le département 75
        normalized_address = re.sub(r'(paris) \d{2}$', r'\1', normalized_address, flags=re.IGNORECASE)
    
    return normalized_address

FONCTION DE MATCHING

In [10]:
def test_match(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['code_departement']==department].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['code_departement']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        #On distingue les adresses uniques et les adresses en doublons pour avoir deux catégories d'adresses et trouver celles communes aux deux bases
        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        # On regarde les adresses communes
        commun = set_final.intersection(set_final2)

        vf_dept['surface_reelle_bati'] = pd.to_numeric(
            vf_dept['surface_reelle_bati'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            # On crée des sous dataframe contenant les lignes avec les mêmes adresses
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            # Boucle sur les rangs des sous dataframe
            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['surface_reelle_bati']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 5% et parmi les lignes avec des surfaces inf au seuil si jamais la valeur foncière est supérieure à celle de la ligne d'avant on la conserve pour avoir la plus grande
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        valeur_fonciere = pd.to_numeric(str(row1.get('valeur_fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df.drop_duplicates(subset=['Date_établissement_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'valeur_fonciere', 'Etiquette_DPE'])


In [66]:
def compiler_dataframes(*dfs):
    # Concatène tous les DataFrames passés en argument
    df_compilé = pd.concat(dfs, ignore_index=True)
    # Supprime les doublons
    df_compilé = df_compilé.drop_duplicates()
    return df_compilé

Fonction pour mettre en forme les fichiers valeurs foncières.

In [12]:
def format_vf(vf):
    # On crée la colonne adresse
    vf['Adresse'] = vf['adresse_numero'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['adresse_nom_voie'].fillna('').astype(str) + " " + \
                vf['code_postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['nom_commune'].fillna('').astype(str)

    #On formate
    vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

    vf = vf[vf['code_departement'].notna()]
    vf['code_departement'] = vf['code_departement'].astype(str)

    return vf

Fonction pour mettre en forme le fichier DPE. 

In [20]:
def format_dpe(df,annee):
    df['Date_établissement_DPE'] = pd.to_datetime(df['Date_établissement_DPE'], errors='coerce')
    df = df[df['Date_établissement_DPE'].dt.year == annee].copy()
    df = df[df['N°_département_(BAN)'].notna()]
    df.loc[:, 'N°_département_(BAN)'] = df['N°_département_(BAN)'].astype(str)
    df = df[df['Etiquette_DPE'].isin(['A', 'B', 'C', 'D', 'E', 'F', 'G'])]

    return df

# 2. Année 2019

In [14]:
v2019 = format_vf(vf_2019)

In [21]:
d2019 = format_dpe(DPE, 2019)

In [22]:
match2019 = test_match(v2019,d2019)

Traitement du département : 48
Traitement du département : 2A
Traitement du département : 2B
Traitement du département : 23
Traitement du département : 15
Traitement du département : 55
Traitement du département : 90
Traitement du département : 43
Traitement du département : 09
Traitement du département : 46
Traitement du département : 52
Traitement du département : 27
Traitement du département : 58
Traitement du département : 32
Traitement du département : 05
Traitement du département : 82
Traitement du département : 19
Traitement du département : 70
Traitement du département : 04
Traitement du département : 36
Traitement du département : 65
Traitement du département : 61
Traitement du département : 18
Traitement du département : 12
Traitement du département : 10
Traitement du département : 39
Traitement du département : 08
Traitement du département : 89
Traitement du département : 07
Traitement du département : 16
Traitement du département : 41
Traitement du département : 88
Traiteme

In [23]:
print(len(match2019))

59551


In [71]:
match2019.to_csv('final2019.csv',index=False)

# 3. Année 2020

In [24]:
v2020 = format_vf(vf_2020)

In [25]:
d2020 = format_dpe(DPE, 2020)

In [26]:
match2020 = test_match(v2020,d2020)

Traitement du département : 48
Traitement du département : 2B
Traitement du département : 2A
Traitement du département : 23
Traitement du département : 55
Traitement du département : 15
Traitement du département : 90
Traitement du département : 43
Traitement du département : 09
Traitement du département : 52
Traitement du département : 46
Traitement du département : 58
Traitement du département : 32
Traitement du département : 05
Traitement du département : 70
Traitement du département : 82
Traitement du département : 19
Traitement du département : 04
Traitement du département : 65
Traitement du département : 61
Traitement du département : 89
Traitement du département : 18
Traitement du département : 10
Traitement du département : 12
Traitement du département : 08
Traitement du département : 39
Traitement du département : 36
Traitement du département : 41
Traitement du département : 07
Traitement du département : 16
Traitement du département : 03
Traitement du département : 88
Traiteme

In [72]:
match2020.to_csv('final2020.csv',index=False)

# 4. Année 2021

In [28]:
v2021 = format_vf(vf_2021)

In [29]:
d2021 = format_dpe(DPE,2021)

In [30]:
match2021 = test_match(v2021,d2021)

Traitement du département : 48
Traitement du département : 2B
Traitement du département : 2A
Traitement du département : 55
Traitement du département : 15
Traitement du département : 23
Traitement du département : 90
Traitement du département : 43
Traitement du département : 09
Traitement du département : 46
Traitement du département : 52
Traitement du département : 58
Traitement du département : 32
Traitement du département : 05
Traitement du département : 70
Traitement du département : 19
Traitement du département : 04
Traitement du département : 82
Traitement du département : 08
Traitement du département : 18
Traitement du département : 10
Traitement du département : 61
Traitement du département : 36
Traitement du département : 12
Traitement du département : 65
Traitement du département : 40
Traitement du département : 39
Traitement du département : 16
Traitement du département : 41
Traitement du département : 07
Traitement du département : 03
Traitement du département : 89
Traiteme

In [73]:
match2021.to_csv('final2021.csv',index=False)

# 5. Année 2022

In [32]:
v2022 = format_vf(vf_2022)

In [33]:
d2022 = format_dpe(DPE,2022)

In [34]:
match2022 = test_match(v2022,d2022)

Traitement du département : 14
Traitement du département : 58
Traitement du département : 44
Traitement du département : 68
Traitement du département : 71
Traitement du département : 62
Traitement du département : 74
Traitement du département : 17
Traitement du département : 83
Traitement du département : 34
Traitement du département : 33
Traitement du département : 91
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 92
Traitement du département : 59
Traitement du département : 94
Traitement du département : 75
Traitement du département : 50
Traitement du département : 37
Traitement du département : 13
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 29
Traitement du département : 63
Traitement du département : 36
Traitement du département : 95
Traitement du département : 76
Traitement du département : 78
Traitement du département : 38
Traiteme

In [36]:
print(len(match2022))

175491


In [74]:
match2022.to_csv('final2022.csv',index=False)

# 6. Année 2023

In [37]:
v2023 = format_vf(vf_2023)

In [38]:
d2023 = format_dpe(DPE, 2023)

In [39]:
match2023 = test_match(v2023,d2023)

Traitement du département : 84
Traitement du département : 32
Traitement du département : 59
Traitement du département : 34
Traitement du département : 56
Traitement du département : 44
Traitement du département : 76
Traitement du département : 35
Traitement du département : 93
Traitement du département : 49
Traitement du département : 69
Traitement du département : 77
Traitement du département : 83
Traitement du département : 75
Traitement du département : 22
Traitement du département : 16
Traitement du département : 78
Traitement du département : 73
Traitement du département : 50
Traitement du département : 92
Traitement du département : 80
Traitement du département : 11
Traitement du département : 67
Traitement du département : 30
Traitement du département : 14
Traitement du département : 60
Traitement du département : 64
Traitement du département : 94
Traitement du département : 29
Traitement du département : 28
Traitement du département : 91
Traitement du département : 13
Traiteme

In [40]:
print(len(match2023))

97419


In [75]:
match2023.to_csv('final2023.csv',index=False)

# 7. Année 2024

In [44]:
print(len(vf_2024))

1566643


In [41]:
v2024 = format_vf(vf_2024)

In [42]:
d2024 = format_dpe(DPE,2024)

In [45]:
print(len(d2024))

9


In [43]:
match2024 = test_match(v2024,d2024)

Traitement du département : 65
Traitement du département : 06


In [65]:
print(len(match2024))

0


# 8. Matching des 6 bases de données

In [67]:
result = compiler_dataframes(match2019,match2020,match2021,match2022,match2023,match2024)

In [68]:
print(len(result))

512824
